In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import json
import re
import string
from IPython.display import display, HTML
import keras
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, losses, callbacks

2024-12-23 12:14:37.466302: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-23 12:14:37.466351: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-23 12:14:37.467386: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("TensorFlow is using the GPU")
else:
    print("TensorFlow is not using the GPU")


TensorFlow is not using the GPU


In [2]:
MAXLEN = 80
VOCAB_SIZE = 10000
EMBEDDING_DIM = 256
N_HEADS = 2
KEY_DIM = 256
FEED_FORWARD_DIM = 256

## Prepare the dataset

In [3]:
wine_data = pd.read_csv("Data/winemag-data-130k-v2.csv")
wine_data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [4]:
wine_data.to_json("Data/winemag-data-130k-v2.json", orient="records", indent=4)

In [5]:
# Load the full dataset
with open("Data/winemag-data-130k-v2.json") as json_data:
    wine_data = json.load(json_data)

In [6]:
filtered_data = [
    "wine_data: "
    + x["country"] + " : "
    + x["province"] + " : "
    + x["variety"] + " : "
    + x["description"] + " : "
    for x in wine_data
    if x["country"] is not None
    and x["province"] is not None
    and x["variety"] is not None
    and x["description"] is not None
]

In [7]:
number_wines = len(filtered_data)
print(f"{number_wines} load_dataset")

129907 load_dataset


In [8]:
sample = filtered_data[25]
print(sample)

wine_data: US : California : Pinot Noir : Oak and earth intermingle around robust aromas of wet forest floor in this vineyard-designated Pinot that hails from a high-elevation site. Small in production, it offers intense, full-bodied raspberry and blackberry steeped in smoky spice and smooth texture. : 


## tokenize the dataset

In [9]:
def pad_punt(s):
    s = re.sub(f"([{string.punctuation}, '\n'])", r" \1 ", s)
    s =  re.sub(" +"," ", s)
    return s

In [10]:
text_data = [pad_punt(x) for x in filtered_data]

In [11]:
sample=text_data[25]

In [12]:
text_ds = tf.data.Dataset.from_tensor_slices(text_data).batch(32).shuffle(1000)

2024-12-17 08:05:17.788925: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-17 08:05:17.828555: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-17 08:05:17.828756: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [13]:
vectorize = layers.TextVectorization(
                    standardize = "lower",
                    max_tokens=VOCAB_SIZE,
                    output_mode = "int",
                    output_sequence_length= MAXLEN + 1)

In [14]:
vectorize.adapt(text_ds)
vocab = vectorize.get_vocabulary()

In [15]:
def prep_input(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentence = vectorize(text)
    x = tokenized_sentence[:,:-1]
    y = tokenized_sentence[:, 1:]
    return x, y
train_ds = text_ds.map(prep_input)

## AttentionHead

In [16]:
# layers.MultiHeadAttention(num_heads = 4,
#                           key_dim = 128,
#                           value_dim = 64,
#                           output_shape = 128
#                             )

In [17]:
def Causal_Attn_Mask(batch_size, n_dest, n_src, dtype):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat([tf.expand_dims(batch_size, - 1), tf.constant([1,1],
                     dtype=tf.int32)], 0)
    return tf.tile(mask, mult)
np.transpose(Causal_Attn_Mask(1,10,10, dtype=tf.int32)[0])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

## TransformerBlock

In [18]:
class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, key_dim, ff_dim, embed_dim, rate= 0.1):
        super().__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.rate = rate
        self.attn = layers.MultiHeadAttention(num_heads, key_dim, output_shape=embed_dim)
        self.dropout1 = layers.Dropout(self.rate)
        self.layerNorm = layers.LayerNormalization(epsilon=1e-6)
        self.ffn = keras.Sequential(
            [layers.Dense(self.ff_dim, activation="relu"), layers.Dense(self.embed_dim),]
        )
        self.dropout2 = layers.Dropout(self.rate)
        self.layerNorm2 = layers.LayerNormalization(epsilon=1e-6)
        
    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        sequence_length = input_shape[1]
        causal_mask = Causal_Attn_Mask(batch_size, sequence_length, sequence_length, tf.bool)
        attention_output, attention_scores = self.attn(inputs,
                                            inputs,
                                            attention_mask = causal_mask,
                                            return_attention_scores = True)
        attention_output = self.dropout1(attention_output)
        out_1 = self.layerNorm(inputs + attention_output)
        ffn = self.ffn(out_1)
        ffn_out = self.dropout2(ffn)
        return (self.layerNorm2(out_1 + ffn_out), attention_scores)
    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "key_dim": self.key_dim,
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "ff_dim": self.ff_dim,
                "rate": self.rate,
            }
        )
        return config

In [19]:
# TransformerEncoder.call(
#     inputs,
#     padding_mask=None,
#     attention_mask=None,
#     training=None,
#     return_attention_scores=False,
# )


## PositioningEmbedding

In [20]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self,maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.maxlen = maxlen
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(input_dim = vocab_size, output_dim = embed_dim)
        self.pos_emb = layers.Embedding(input_dim = maxlen, output_dim = embed_dim)
        
    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "max_len": self.maxlen,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

In [21]:
inputs = layers.Input(shape=(None,), dtype=tf.int32)
x = TokenAndPositionEmbedding(MAXLEN, VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x, attention_scores = TransformerBlock(N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM)(x)
outputs = layers.Dense(VOCAB_SIZE, activation ="softmax")(x)
gpt = models.Model(inputs = inputs, outputs=[outputs, attention_scores])
gpt.compile("adam", loss=[losses.SparseCategoricalCrossentropy(), None])

In [22]:
gpt.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddi  (None, None, 256)         2580480   
 ng (TokenAndPositionEmbedd                                      
 ing)                                                            
                                                                 
 transformer_block (Transfo  ((None, None, 256),       658688    
 rmerBlock)                   (None, 2, None, None))             
                                                                 
 dense_2 (Dense)             (None, None, 10000)       2570000   
                                                                 
Total params: 5809168 (22.16 MB)
Trainable params: 5809168 (22.16 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [23]:
checkpoint_path = "./checkpoint/checkpoints.ckpt"
ModelcheckPoint = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_best_only=True,
                                                 save_freq="epoch",
                                                 verbose=0,
                                                 save_weights_only=True)
EarlyStopping = keras.callbacks.EarlyStopping(monitor="loss",
                                             verbose=0,
                                             mode="auto",
                                             patience=3,
                                             min_delta=0.01,
                                             )
tensorboard = keras.callbacks.TensorBoard(log_dir="./logs")

In [24]:
gpt.fit(train_ds,
        epochs= 15,
       callbacks=[ModelcheckPoint, EarlyStopping, tensorboard])

Epoch 1/15


2024-12-17 08:05:51.478177: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f2020612d90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-12-17 08:05:51.478224: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-12-17 08:05:51.484373: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-17 08:05:52.004857: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8903
I0000 00:00:1734422752.151215   12067 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4060/4060 [==============================] - 144s 33ms/step - loss: 2.2507 - dense_2_loss: 2.2507
Epoch 2/15
4060/4060 [==============================] - 98s 24ms/step - loss: 1.9558 - dense_2_loss: 1.9558
Epoch 3/15
4060/4060 [==============================] - 98s 24ms/step - loss: 1.8861 - dense_2_loss: 1.8861
Epoch 4/15
4060/4060 [==============================] - 98s 24ms/step - loss: 1.8446 - dense_2_loss: 1.8446
Epoch 5/15
4060/4060 [==============================] - 96s 24ms/step - loss: 1.8142 - dense_2_loss: 1.8142
Epoch 6/15
4060/4060 [==============================] - 96s 24ms/step - loss: 1.7908 - dense_2_loss: 1.7908
Epoch 7/15
4060/4060 [==============================] - 98s 24ms/step - loss: 1.7721 - dense_2_loss: 1.7721
Epoch 8/15
4060/4060 [==============================] - 93s 23ms/step - loss: 1.7564 - dense_2_loss: 1.7564
Epoch 9/15
4060/4060 [==============================] - 94s 23ms/step - loss: 1.7426 - dense_2_loss: 1.7426
Epoch 10/15
4060/4060 [===============

In [25]:
# import tensorflow as tf

# # Check if GPU is available
# if tf.config.list_physical_devices('GPU'):
#     print("TensorFlow is using GPU.")
# else:
#     print("TensorFlow is not using GPU.")